<a href="https://colab.research.google.com/github/armanheydari/Advance-Deep-Learning_Winter-2024/blob/main/Assignment3/cmpt489_828_a3_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CMPT 489/828 Assignment 3**

Follow the instructions in this notebook and complete the missing code.

**NOTE: Do Not Change Any Provided Code or Given Variable Names!**


Feel free to reuse any code from previous assignments.

Use torch.nn module to implement your network.

Use the validation set to find the best hyperparameters.

In [ ]:
# TODO: your imports
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import transforms
import torchvision.models as models
from torchvision.models import ResNet18_Weights

# select gpu if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def create_data_loader(batch_size):
  # TODO: create your data loader, use entire CIFAR-10
  # Split the provided CIFAR-10 train set (50,000 images) into your train and val sets
  # Use the first 40,000 images as your train set and the remaining 10,000 images as val set
  # Use all 10,000 images in the provided test set as your test set

  train_id = list(range(40000))
  val_id = list(range(40000, 50000))
  test_id = list(range(10000))

  # convert to tensor, normalize
  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
  ])
  # load CIFAR-10 dataset with pytorch
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

  # subset dataset and create dataloader with batch_size
  train_sub_set = torch.utils.data.Subset(trainset, train_id)
  val_sub_set = torch.utils.data.Subset(trainset, val_id)
  test_sub_set = torch.utils.data.Subset(testset, test_id)

  train_loader = torch.utils.data.DataLoader(train_sub_set, batch_size=batch_size, shuffle=True)
  val_loader = torch.utils.data.DataLoader(val_sub_set, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_sub_set, batch_size=batch_size, shuffle=True)

  return train_loader, val_loader, test_loader

In [ ]:
class ResidualBlock(nn.Module):
  # TODO: implement a residual block with skip connection


In [ ]:
class ResNet(nn.Module):
  # TODO: implement a ResNet with num_block residual blocks
  def __init__(self, num_block)



In [ ]:
def init_train_var(model):
  # TODO: create your criterion, optimizer (use SGD)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters())
  return criterion, optimizer

In [ ]:
def train(train_loader, val_loader, batch_size):
  # TODO: implement your training loop, save your best model at the end
  model = ResNet(4)
  criterion, optimizer = init_train_var(model)

  epochs = 100
  val_accuracy = 0
  for epoch in range(epochs):
    model.train()
    train_correct, val_correct = 0, 0
    for x, y in train_loader:
      x = x.to(device)
      y = y.to(device)
      optimizer.zero_grad()
      prediction = model.forward(x)
      loss = criterion(prediction, y)
      loss.backward()
      optimizer.step()
      train_correct += (prediction.argmax(1) == y).type(torch.float).sum().item()

    # switch off autograd for validation
    with torch.no_grad():
      # set the model in evaluation mode
      model.eval()
      for x, y in val_loader:
        x = x.to(device)
        y = y.to(device)
        prediction = model.forward(x)
        val_correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
    # calculate the training and validation accuracy
    train_accuracy = train_correct / len(train_loader.dataset)
    val_accuracy_new = val_correct / len(val_loader.dataset)
    print(f'In epoch {epoch+1} the training accuracy is {train_accuracy} and the validation accuracy is {val_accuracy_new}')
    if val_accuracy>val_accuracy_new:
      # Save entire model
      model_path = 'Resnet18_Cifar10_Arman.pth'
      torch.save(model, model_path)
      break
    val_accuracy = val_accuracy_new
  return model_path

In [ ]:
def test(model_path, test_loader):
  # TODO: test function for your trained model , load your best model
  model = torch.load(model_path)
  # switch off autograd for validation
  with torch.no_grad():
    # set the model in evaluation mode
    model.eval()
    test_correct = 0
    for x, y in test_loader:
      x = x.to(device)
      y = y.to(device)
      prediction = model.forward(x)
      test_correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
  # calculate the training and validation accuracy
  test_accuracy = test_correct / len(test_loader.dataset)
  print(f"model's test accuracy is {test_accuracy}")

In [ ]:
# TODO: train your model and save the best model
batch_size = 100
train_loader, val_loader, test_loader = create_data_loader(batch_size)
model_path = train(train_loader, val_loader, batch_size)

In [ ]:
# TODO: report test performance of best model
test(model_path, test_loader)